In [55]:
%matplotlib inline
from matplotlib import pyplot as plt
from ipycanvas import Canvas
import copy
import numpy as np
from scipy import ndimage as ndi

canvas_size = 32
brush_size = 2

image_size = 28

canvas = Canvas(width = canvas_size, height = canvas_size, sync_image_data=True)

canvas.layout.width = '400px'
canvas.layout.height = 'auto'

drawing = False

def handle_mouse_down(x, y):
    global drawing
    drawing = True
    canvas.fill_rect(x, y, brush_size, height=None)

def handle_mouse_move(x, y):
    if drawing == True:
        canvas.fill_rect(x, y, brush_size, height=None)
    else:
        pass

def handle_mouse_up(x, y):
    global drawing
    drawing = False
    

canvas.on_mouse_down(handle_mouse_down)
canvas.on_mouse_move(handle_mouse_move)
canvas.on_mouse_up(handle_mouse_up)

canvas.stroke_style = 'red'
canvas.stroke_rect(0, 0, canvas_size, height=None)

canvas

Canvas(height=32, layout=Layout(height='auto', width='400px'), sync_image_data=True, width=32)

In [69]:
def grayscale(image_array): #input is a 4-channel image in the form of an array with shape (x_dim, y_dim, n_channels)
    
    grayscale_image_array = copy.deepcopy(image_array)
    
    grayscale_image_array = np.ravel(grayscale_image_array) #unrolls the array into a row vector
    grayscale_image_array = grayscale_image_array[3:len(grayscale_image_array):4] #selects only the values of the channel of interest
    grayscale_image_array = np.reshape(grayscale_image_array, (image_size, image_size)) #repacks the image in a 2D array
    
    return grayscale_image_array

def center(image_array): #input is a grayscale image in the form of an array with shape (x_dim, y_dim)
    
    center_of_mass = ndi.measurements.center_of_mass(image_array) #calculates the center of mass of the image
    roll_factors = (round((image_size)/2 - center_of_mass[0]), #calculates the displacement in the x and y
                    round((image_size)/2 - center_of_mass[1])) #directions to center the image
    
    centered_image_array = copy.deepcopy(image_array)
    
    centered_image_array = np.roll(centered_image_array, roll_factors[1], axis = 1)
    centered_image_array = np.roll(centered_image_array, roll_factors[0], axis = 0)
    
    return centered_image_array

def prepare_NN_input(image_array):
    
    image_array = grayscale(image_array)
    image_array = center(image_array)
    
    image_array = image_array.astype('float32') / 255 #converts the pixel value range from 0-255 to 0-1
    image_array = image_array.reshape((1,image_size**2)) #unrolls the 2D array in a column vector
    
    return image_array
    

In [70]:
canvas_array = canvas.get_image_data(2, 2, image_size, image_size)
nn_input_array = prepare_NN_input(canvas_array)

canvas_drawing_predition = network.predict(nn_input_array)

np.argmax(canvas_drawing_predition)

3

In [6]:
from keras import models
from keras import layers
from keras import optimizers
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

network.fit(train_images, train_labels, epochs=5, batch_size=128)

Using TensorFlow backend.
2022-01-01 17:50:27.152403: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-01 17:50:27.155934: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 4. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/5
60000/60000 [==============================] - 12s 194us/step - loss: 0.2581 - accuracy: 0.9253
Epoch 2/5
60000/60000 [==============================] - 11s 181us/step - loss: 0.1036 - accuracy: 0.9683
Epoch 3/5
60000/60000 [==============================] - 11s 183us/step - loss: 0.0676 - accuracy: 0.9797
Epoch 4/5
60000/60000 [==============================] - 11s 177us/step - loss: 0.0492 - accuracy: 0.9854
Epoch 5/5
60000/60000 [==============================] - 11s 187us/step - loss: 0.0372 - accuracy: 0.9885
